In [1]:
import glob
import os
import pandas as pd
import numpy as np
from numpy import split
from numpy import array
from math import sqrt
from pandas import read_csv
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from pathlib import Path
from numpy import hstack
from pandas import read_csv
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier




def split_sequences(sequences, n_steps, n_out):
	trainx, trainy = list(), list()
	testx, testy = list(), list()
	testsplit = int((len(sequences)) *.20) # change percentage for test here
	train, test = sequences[0:-(testsplit)], sequences[-(testsplit):]
   

	for i in range(len(train)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(train)-(1+n_out):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = train[i:end_ix, :-1], train[end_ix+n_out, -1:]          
		tk1 = train[i:end_ix+1+n_out, :-1]
		first = tk1[0,0]
		last =  tk1[n_steps+n_out,0]        
		if first==last:
			trainy.append(seq_y)
			trainx.append(seq_x[:,2:])
		else:
			None 
            
	for i in range(len(test)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(test)-(1+n_out):  #leave # of x rows on bottom for process to stop
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = test[i:end_ix, :-1], test[end_ix+n_out, -1:]    #get batch of n_steps for X, batch of y data is plus 1 (or x) vertically      
		tk1 = test[i:end_ix+1+n_out, :-1]     # get the batch above plus 1 (or x) more rows to test if belong to same ticker
		first = tk1[0,0]
		last =  tk1[n_steps+n_out,0]        
		if first==last:
			testy.append(seq_y)
			testx.append(seq_x[:,2:])
		else:
			None
	return array(trainx), array(trainy), array(testx), array(testy)



# the path to csv file directory
mycsvdir = 'C:/Users/singh/Documents/rawdata/test'

# get all the csv files in that directory (assuming they have the extension .csv)
csvfiles = glob.glob(os.path.join(mycsvdir, '*.csv'))

# loop through the files and read them in with pandas
dataframes = []  # a list to hold all the individual pandas DataFrames
for csvfile in csvfiles:
    
    #read data, clean hyphen and text, remove date columns, convert string object to float, remove redundant columns, average price, remove high/low price columns, append to dataframe
    rawdata = pd.read_csv(csvfile)
    dataset = rawdata.iloc[18:230, 33:153]
    dft = dataset.T
    #breakpoint()
    base, ext = os.path.splitext(Path(csvfile).name)
    dft[18] = base
    dft = dft.replace('-', 0)
    dft = dft.replace('Negative Tangible Equity', 0)
    dft = dft.replace('No Debt', 0)
    dataframes.append(dft)
    
# concatenate them all together
result = pd.concat(dataframes, ignore_index=True)
result = result.drop(columns=[21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,73,79,80,82,84,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,217,218,219,220,221,222,225,226,227,228,229])
result[[42,59,60,61,62,63,64,65,66,67,68,69,70,71,72,74,75,76,77,78,81,83,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,197,215,216,223,224]] = result[[42,59,60,61,62,63,64,65,66,67,68,69,70,71,72,74,75,76,77,78,81,83,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,197,215,216,223,224]].astype('float32')
result[230] = result[[215, 216]].mean(axis=1)
result = result.drop(columns=[215, 216])
result = result[result[230] > 0]
result = result.dropna()
result[231] = result[230].pct_change().astype('float32')
result[231].fillna(0, inplace=True)

result[231] = np.where(result[231]>.05, 1, result[231])
#result[231] = np.where(result[231].between(.25,1), 6, result[231])
#result[231] = np.where(result[231].between(-.05,.05), 0, result[231])

result[231] = np.where(result[231]<.05, -1, result[231])
#result[231] = np.where(result[231].between(-1,-.25), -6, result[231])
#result[231] = np.where(result[231].between(-.05,-0.00000000001), -1, result[231])

result.to_csv('all.csv')
result = result.drop(columns=[230])

# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
	# nonlinear models
	models['knn'] = KNeighborsClassifier(n_neighbors=14)
	models['cart'] = DecisionTreeClassifier()
	models['svm'] = SVC()
	models['bayes'] = GaussianNB()
	# ensemble models
	models['bag'] = BaggingClassifier(n_estimators=400)
	models['rf'] = RandomForestClassifier(n_estimators=400)
	models['et'] = ExtraTreesClassifier(n_estimators=400)
	models['gbm'] = GradientBoostingClassifier(n_estimators=400)
	print('Defined %d models' % len(models))
	return models

# evaluate a single model
def evaluate_model(trainX, trainy, testX, testy, model):
	# fit the model
	model.fit(trainX, trainy)
	# make predictions
	yhat = model.predict(testX)
	# evaluate predictions
	accuracy = accuracy_score(testy, yhat)
	return accuracy * 100.0

# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(trainX, trainy, testX, testy, models):
	results = dict()
	for name, model in models.items():
		# evaluate the model
		results[name] = evaluate_model(trainX, trainy, testX, testy, model)
		# show process
		print('>%s: %.3f' % (name, results[name]))
	return results

# print and plot the results
def summarize_results(results, maximize=True):
	# create a list of (name, mean(scores)) tuples
	mean_scores = [(k,v) for k,v in results.items()]
	# sort tuples by mean score
	mean_scores = sorted(mean_scores, key=lambda x: x[1])
	# reverse for descending order (e.g. for accuracy)
	if maximize:
		mean_scores = list(reversed(mean_scores))
	print()
	for name, score in mean_scores:
		print('Name=%s, Score=%.3f' % (name, score))

n_steps = 3
n_out = 0    # 0 is t+1, 1 is next one t+2
# convert into input/output
trainX, trainy, testX, testy  = split_sequences(result.values, n_steps, n_out)

print(trainX.shape)
print(trainy.shape)
print(testX.shape)
print(testy.shape)
print(n_out)

	# flatten X
trainX = trainX.reshape((trainX.shape[0], trainX.shape[1] * trainX.shape[2]))
trainX = trainX.astype('float32')
trainX = trainX.astype('int')
testX = testX.reshape((testX.shape[0], testX.shape[1] * testX.shape[2]))
testX = testX.astype('float32')
testX = testX.astype('int')
	# flatten y
trainy, testy = trainy[:,0], testy[:,0]
trainy = trainy.astype('int')
testy = testy.astype('int')

r_scaler = preprocessing.Normalizer()
trainX = r_scaler.fit_transform(trainX)
testX = r_scaler.fit_transform(testX)

#r_scaler = preprocessing.StandardScaler()
#trainX = r_scaler.fit_transform(trainX)
#testX = r_scaler.fit_transform(testX)

# get model list
models = define_models()
# evaluate models
results = evaluate_models(trainX, trainy, testX, testy, models)
# summarize results
summarize_results(results)

Using TensorFlow backend.


(1682, 3, 122)
(1682, 1)
(412, 3, 122)
(412, 1)
0
Defined 8 models
>knn: 59.223
>cart: 52.184
>svm: 64.563
>bayes: 54.369
>bag: 59.951
>rf: 58.738
>et: 59.709
>gbm: 58.252

Name=svm, Score=64.563
Name=bag, Score=59.951
Name=et, Score=59.709
Name=knn, Score=59.223
Name=rf, Score=58.738
Name=gbm, Score=58.252
Name=bayes, Score=54.369
Name=cart, Score=52.184
